To dos:
- create a dataset with age and lon, lat and download it - CHECK
- specify a new location to save images on the server -  CHECK
- loop over the dataset - CHECK
 - plug in lat and lon into the api code below - CHECK
 - extract all images for that location - CHECK
 - name image according to age and its unique code after the last - CHECK

CNN
- rewatch simple CNN video how to extract age from name file - CHECK
- run a simple CNN and compare results - CHECK
- how to boost perfromance - HALF-CHECK - wait for reply from Nikola and start reading papers to do it yourself.
- How to increase image quality and coverage? - HALF-CHECK - wait for Nikolas reply on a code basis, read other papers on how they implement it.

Extent dataset:
- access Mapillary data either through API or the download you did
- repeat above excercise
- compare R2
- compare coverage of images between the two - are the duplicates?

Integration
- add a building ID to the kartaview key 
- Download images for buildings that do not have transaction data and train the algorithm on it
- use the images that have transaction data as test set - CAREFUL THO DUE TO SELECTION ISSUES.
- save the age predictions from this first step and add them as prediction variable into the matched GIS-RCA dataset as a variable using the building ID as key
- rerun the xgboost model on it and see how it performs
- in the best scenario, you can develop a multi-modal model that optimises the relational and image data together in one deep learning model, but this is REALLY OPTIONAL
- OPTIONAL: Does it make sense to run a computer vision excercise to predict architectural style and feed that into the relational data?

## Imports

In [1]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

## Data

In [21]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP',
    'ALL': '/workspace/workspace/ufo-prediction/image_data_ID'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=100000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Directory /workspace/workspace/ufo-prediction/image_data_FRA already exists.
Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv
Empty DataFrame
Columns: [lon, lat, age_right, id]
Index: []


In [22]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_FRA
Empty DataFrame
Columns: [lon, lat, age_right, id]
Index: []


In [23]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data_ID': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

for index, row in kartaview_keys.iterrows():
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Save the updated DataFrame to a CSV file
    kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))


Images will be saved in: /workspace/workspace/ufo-prediction/image_data_FRA
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv
Number of buildings remaining:  0
